In [ ]:
!pip3 install transformers sentencepiece pandas gensim

In [ ]:
import pandas as pd 
import numpy as np 
import torch
from transformers import AutoTokenizer, AutoModel
import gensim
import csv
import gc

In [ ]:
cols = ['title', 'description']
data = pd.read_csv('/storage/data/simple_features.csv', chunksize=100000)

In [ ]:
for i, chunk in enumerate(data):
    chunk = chunk[cols].fillna('продать')
    for col in cols:
        chunk[col].to_csv('/storage/data/{}.txt'.format(col), header=False, index=False, sep=' ', encoding='utf-8',
        quoting=csv.QUOTE_NONE, escapechar=' ', mode='w' if i == 0 else 'a')
data = None
gc.collect()

In [ ]:
for col in cols:
  model = gensim.models.doc2vec.Doc2Vec(vector_size=100 if col == 'title' else 300, 
                                        window = 3 if col == 'title' else 5,
                                        min_count=10, epochs=30, workers=8)
  model.build_vocab(corpus_file='/storage/data/{}.txt'.format(col))
  model.train(corpus_file='/storage/data/{}.txt'.format(col),
   total_examples=model.corpus_count, epochs=model.epochs, total_words=model.corpus_count)
  model.save('/storage/data/doc2vec_{}'.format(col))
  gc.collect()